# BDLE : Graphes en DataFrame

## Préparation

*   ***Vérifier que des ressources*** de calcul sont allouées à votre notebook est 
connecté (cf RAM de disque indiqués en haut à droite) . Sinon cliquer sur le bouton connecter pour obtenir des ressources.

*   ***Créer le répertoire*** pour stocker les fichiers nécessaires sur votre google 
drive (donnez l'autorisation au notebook d'accéder à votre drive lorsque c'est demandé). *Ajuster le nom de votre dossier* : **MyDrive/ens/bdle/GraphDF.**

In [ ]:
import os
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

drive_dir = "/content/drive/MyDrive/ens/bdle/GraphDF/"
os.makedirs(drive_dir, exist_ok=True)
os.listdir(drive_dir)

Mounted at /content/drive


['data.csv.gz', 'meta.csv.gz']

***Installer pyspark et findspark :***

In [ ]:
!pip install -q pyspark
!pip install -q findspark

     |████████████████████████████████| 281.4 MB 27 kB/s 
     |████████████████████████████████| 198 kB 61.3 MB/s 


***Démarrer la session spark:***

In [ ]:
import os
# !find /usr/local -name "pyspark"
os.environ["SPARK_HOME"] = "/usr/local/lib/python3.7/dist-packages/pyspark"
os.environ["JAVA_HOME"] = "/usr"

In [ ]:
# Principaux import
import findspark
from pyspark.sql import SparkSession 
from pyspark import SparkConf  

# pour les dataframe et udf
from pyspark.sql import *  
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *

# pour le chronomètre
import time

# initialise les variables d'environnement pour spark
findspark.init()

# Démarrage session spark 
# --------------------------
def demarrer_spark():
  local = "local[*]"
  appName = "TP"
  configLocale = SparkConf().setAppName(appName).setMaster(local).\
  set("spark.executor.memory", "6G").\
  set("spark.driver.memory","6G").\
  set("spark.sql.catalogImplementation","in-memory")
  
  spark = SparkSession.builder.config(conf = configLocale).getOrCreate()
  sc = spark.sparkContext
  sc.setLogLevel("ERROR")
  
  spark.conf.set("spark.sql.autoBroadcastJoinThreshold","-1")

  # On ajuste l'environnement d'exécution des requêtes à la taille du cluster (4 coeurs)
  spark.conf.set("spark.sql.shuffle.partitions","4")    
  print("session démarrée, son id est ", sc.applicationId)
  return spark
spark = demarrer_spark()

session démarrée, son id est  local-1643314514352


In [ ]:
# on utilise 8 partitions au lieu de 200 par défaut
spark.conf.set("spark.sql.shuffle.partitions", "8")
print("Nombre de partitions utilisées : ", spark.conf.get("spark.sql.shuffle.partitions"))

Nombre de partitions utilisées :  8


## Données synthétiques sur la musique

##### Description des données: 

- Fichier **data.csv** : contient des informations sur des chansons (trackId) interprétées par des artistes (artistId) et   écoutées par des utilisateurs (userId) à une date donnée par une estampille temporelle timestamp. Contient 260664 lignes.
- Fichier **meta.csv**: contient les noms (champ 'Name') des chansons si type==track ou des artistes si type==artist.
  'Artist' est le nom de l'artiste qui interprète la chanson si type==track ou le nom de l'artiste (même valeur que 'Name') si track==artist. 'Id' est l'identifiant d'une chanson ou d'un artiste. Contient 44319 lignes.

In [ ]:
# URL du dossier contenant des fichiers de données (data.csv et meta.csv) utiles pour le TP
# ---------------------------------------------------------------------------
# en cas de problème avec le téléchargement des datasets, aller directement sur l'URL ci-dessous
PUBLIC_DATASET_URL = "https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4" 
PUBLIC_DATASET=PUBLIC_DATASET_URL + "/download?path="

print("URL pour les données: ", PUBLIC_DATASET_URL)

URL pour les données:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4


## Lecture des données 




In [ ]:
import os
from urllib import request

def load_file(file,dir):
  if(os.path.isfile(drive_dir+file)):
    print(file, "is already stored")
  else:
    url = PUBLIC_DATASET + "/"+ dir + "/" + file
    print("downloading from URL: ", url, "save in : " + drive_dir   + file)
    request.urlretrieve(url , drive_dir + file)

load_file("data.csv.gz", "musique")
load_file("meta.csv.gz", "musique")

# Liste des fichiers téléchargés
print("Fichiers musique téléchargés:")
os.listdir(drive_dir)

data.csv.gz is already stored
meta.csv.gz is already stored
Fichiers musique téléchargés:


['data.csv.gz', 'meta.csv.gz']

In [ ]:
#Le dossier contenant les fichiers csv importés:
DATASET_DIR="/content/drive/MyDrive/ens/bdle/GraphDF"

#==============
# Données 
#==============
schema = """
          userId LONG, 
          trackId LONG, 
          artistId LONG,
          timestamp LONG
        """
print("Lecture du fichier: ", DATASET_DIR+"/data.csv.gz")
data = spark.read.format("csv").option("header", "true").schema(schema) \
            .load(DATASET_DIR+"/data.csv.gz").persist()
data.show(5)
data.count() #260664


Lecture du fichier:  /content/drive/MyDrive/ens/bdle/GraphDF/data.csv.gz
+------+-------+--------+----------+
|userId|trackId|artistId| timestamp|
+------+-------+--------+----------+
| 13065| 944906|  978428|1501588527|
|101897| 799685|  989262|1501555608|
|215049| 871513|  988199|1501604269|
|309769| 857670|  987809|1501540265|
|397833| 903510|  994595|1501597615|
+------+-------+--------+----------+
only showing top 5 rows



260664

In [ ]:
#==============
# Description des données
#==============
schema = """
          type STRING, 
          Name STRING, 
          Artist STRING,
          Id LONG
        """
print("Lecture du fichier: ", DATASET_DIR+"/meta.csv.gz")
meta = spark.read.format("csv").option("header", "true").schema(schema) \
            .load(DATASET_DIR+"/meta.csv.gz").persist()

meta.show(5)
meta.count() #44319


Lecture du fichier:  /content/drive/MyDrive/ens/bdle/GraphDF/meta.csv.gz
+-----+-----------------+--------------------+------+
| type|             Name|              Artist|    Id|
+-----+-----------------+--------------------+------+
|track|Give Me Some More|Artist: Candy Dulfer|798256|
|track|  Still loven you|   Artist: Scorpions|798258|
|track|             Spez|     Artist: Stromae|798261|
|track| Pachelbels Canon|Artist: David Gar...|798267|
|track|              erg|         Artist: DEV|798274|
+-----+-----------------+--------------------+------+
only showing top 5 rows



44319

## Construction du graphe


In [ ]:
# fonction utilisée par la suite pour calculer les poids des arcs
# df: dataframe, source: nom de la colonne contenant les noeuds source des arcs
# poids: poids initial avant normalisation, n: nombre maximum d'arcs à garder pour chaque source
from pyspark.sql.functions import row_number, sum
from pyspark.sql import Window

def calcul_poids(df, source, poids, n):
    
    window = Window.partitionBy(source).orderBy(col(poids).desc())
    
    # Sert juste à filtrer le nombre d'arc maximum pour chaque source
    filterDF = df.withColumn("row_number", row_number().over(window)) \
        .filter(col("row_number") <= n) \
        .drop(col("row_number")) 
        
    # Somme des poids pour chaque source
    tmpDF = filterDF.groupBy(col(source)).agg(sum(col(poids)).alias("sum_" + poids))
   
   # Normalisation des poids
    finalDF = filterDF.join(tmpDF, source, "inner") \
        .withColumn("norm_" + poids, col(poids) / col("sum_" + poids)) \
        .cache()
        
    return finalDF

 ### Construction des liens pondérés entre utilisateurs et chansons 
* Construire un DataFrame userTrack à partir de data pour stocker les arcs entre utilisateurs et chansons. Pour chaque utilisateur (userId) on ajoute un arc vers une chanson (trackId) avec un poids égal au nombre total de fois que l'utilisateur à écouté la chanson. Utiliser la fonction calcul_poids pour garder pour chaque utilisateur les 100 chansons avec le poids le plus élevé et normaliser les poids des arcs gardés.   
    
* Affichage du résultat: garder uniquement les arcs qui ont les 5 plus grandes valeurs possibles des poids (utilisez la fonction rank() et la fenêtre over(window)). Afficher 20 lignes du résultat, en triant le résultat par ordre décroissant des poids, ensuite par ordre croissant des userId et des artistId.

In [ ]:
from pyspark.sql.functions import col, rank

#nombre de fois que chaque utilisateurs écoute un track
userTrack = data.groupby('userId', 'trackId').agg(first('artistId').alias('artistId'), count('userId').alias('count'))

#calculer le poids final en utilisant calcul_poids
userTrack = calcul_poids(userTrack, 'userId', 'count', 100).persist() 

window = Window.partitionBy('userId').orderBy(col("norm_count").desc())
    
userTrackList = userTrack.withColumn("position", rank().over(window))\
  .where(col("position")<6)\
  .orderBy('userId', 'artistId').select('userId', 'trackId', 'norm_count').take(20)

for val in userTrackList:
   print("%s %s %s" % val)
    
userTrack.count() # résultat: 210675   

10 960353 0.0625
10 855194 0.0625
10 943645 0.0625
10 801772 0.0625
10 958924 0.0625
10 823737 0.0625
10 807650 0.0625
10 824440 0.0625
10 960214 0.0625
10 828318 0.0625
10 955486 0.0625
10 941064 0.0625
10 839649 0.0625
10 956604 0.0625
10 934050 0.0625
10 901153 0.0625
28 837417 0.027777777777777776
28 878347 0.027777777777777776
28 833384 0.027777777777777776
28 952135 0.027777777777777776


210675

### Construction des liens pondérés entre utilisateurs et artistes
* Construire un DataFrame userArtist à partir de data pour stocker les arcs entre utilisateurs et artistes. Pour chaque utilisateur (userId) on ajoute un arc vers un artiste (artistId) avec un poids égal au nombre total de fois que l'utilisateur à écouté des chansons de cet artiste. Utiliser la fonction calcul_poids pour garder pour chaque utilisateur au plus 100 artistes avec le poids le plus élevé et normaliser les poids des arcs gardés.   
    
* Affichage du résultat: garder uniquement les arcs qui ont les 5 plus grandes valeurs possibles des poids (utilisez la fonction rank() et la fenêtre over(window)). 
   Afficher 20 lignes du résultat, en triant le résultat par ordre décroissant des poids, ensuite par ordre croissant des userId et des artistId.

In [ ]:
#poids=nombre de fois qu'un utilisateurs a écouté des morceaux de cet artiste
#regrouper les données par userId et artistId
userArtist = data.groupby('userId', 'artistId').count()

#calculer le poids final en utilisant calcul_poids
userArtist = calcul_poids(userArtist, 'userId', 'count', 100).persist()  

window = Window.partitionBy('userId').orderBy(col("norm_count").desc())
    
userArtistList = userArtist.withColumn("position", rank().over(window))\
  .where(col("position")<6)\
  .orderBy('userId', 'artistId').select('userId', 'artistId', 'norm_count').take(20)

for val in userArtistList:
   print("%s %s %s" % val)
    
userArtist.count() #résultat: 178419

10 969620 0.125
10 970381 0.0625
10 972772 0.25
10 976111 0.0625
10 983989 0.0625
10 984123 0.25
10 987284 0.0625
10 993285 0.0625
10 1001022 0.0625
28 969041 0.027777777777777776
28 970384 0.027777777777777776
28 970511 0.027777777777777776
28 970771 0.027777777777777776
28 971040 0.027777777777777776
28 971515 0.027777777777777776
28 971748 0.027777777777777776
28 972749 0.027777777777777776
28 974503 0.027777777777777776
28 974853 0.027777777777777776
28 977213 0.027777777777777776


178419

### Construction des liens pondérés entre artistes et chansons
* Construire un DataFrame artistTrack à partir de data pour stocker les arcs entre artistes et chansons. Pour chaque artiste (artistId) on ajoute un arc vers une chanson (trackId) avec un poids égal au nombre total de fois que la chanson de cet artiste a été écoutée par tous les utilisateurs. Utiliser la fonction calcul_poids pour garder pour chaque artiste au plus 100 chansons avec le poids le plus élevé et normaliser les poids des arcs gardés.   
    
* Affichage du résultat: garder uniquement les arcs qui ont les 5 plus grandes valeurs possibles des poids (utilisez la fonction rank() et la fenêtre over(window)). 
   Afficher 20 lignes du résultat, en triant le résultat par ordre décroissant des poids, ensuite par ordre croissant des artistId et des trackId.* 

In [ ]:
# poids de l'arc: nombre de fois qu'un track de l'artiste a été écouté par tous les utilisateurs
artistTrack = data.groupby('artistId', 'trackId').count()

#calculer le poids final en utilisant calcul_poids
artistTrack = calcul_poids(artistTrack, 'artistId', 'count', 100).persist() 

window = Window.partitionBy("artistId").orderBy(col("norm_count").desc())
    
artistTrackList = artistTrack.withColumn("position", rank().over(window))\
  .where(col("position")<6)\
  .orderBy('artistId', 'trackId').select('artistId', 'trackId', 'norm_count').take(20)    

for val in artistTrackList:
   print("%s %s %s" % val)
    
artistTrack.count() #résultat: 35408

967986 872118 0.06521739130434782
967986 886535 0.043478260869565216
967986 916802 0.021739130434782608
967986 919902 0.021739130434782608
967986 943482 0.8478260869565217
967988 799810 0.058823529411764705
967988 890687 0.7058823529411765
967988 914158 0.058823529411764705
967988 950807 0.17647058823529413
967990 864814 0.5
967990 877149 0.25
967990 931743 0.25
967992 870986 0.125
967992 914941 0.25
967992 924226 0.625
967993 869415 1.0
967995 800737 0.5
967995 806232 0.5
967997 822911 0.06666666666666667
967997 823263 0.06666666666666667


35408

### Construction des liens pondérés entre chansons
* Construire un DataFrame trackTrack à partir de data pour stocker les arcs entre les chansons. Le poids total d'un arc entre trackId1 et trackId2 est le nombre total d'utilisateurs qui ont écouté à la fois trackId1 et trackId2 dans un laps de temps de 10 minutes (à noter que le graphe est non orienté, trackTrack contient à la fois une entrée pour (trackId1, trackId2) et une entrée pour (trackId2, trackId1)).   
Utiliser la fonction calcul_poids pour garder pour chaque chanson au plus 100 chansons avec le poids le plus élevé et normaliser les poids gardés.   
    
* Affichage du résultat: garder uniquement les arcs qui ont les 5 plus grandes valeurs possibles des poids (utilisez la fonction rank() et la fenêtre over(window)). 
   Afficher 20 lignes du résultat, en triant le résultat par ordre décroissant des poids, ensuite par ordre croissant des artistId et des trackId.

In [ ]:
# Construire les couples de trackId écoutés par le même utilisateur
data2 = data.selectExpr("userId as userId2", "trackId as trackId2", "artistId as artistId2", "timestamp as timestamp2")
trackTrack = data.join(data2, ((col("userId")==col("userId2")) & (col('trackId') != col("trackId2")))).where(abs(col('timestamp')-col('timestamp2')) < 600)\
.select('userId', "trackId", "trackId2")

#pour chaque couple de trackId le nombre d'utilisateurs qui les ont écoutés ensemble
trackTrack = trackTrack.groupby("trackId", "trackId2").count()

#calculer le poids final en utilisant calcul_poids
trackTrack =  calcul_poids(trackTrack, 'trackId', 'count', 100).persist()

window = Window.partitionBy('trackId').orderBy(col("norm_count").desc())

trackTrackList = trackTrack.withColumn("position", rank().over(window))\
    .where(col("position")<6)\
    .orderBy('trackId').select('trackId', "trackId2", 'norm_count').take(20)   

for val in trackTrackList:
   print("%s %s %s" % val)
    
trackTrack.count() #résultat: 136257  

798256 954826 0.5
798256 923706 0.5
798258 810685 0.5
798258 808254 0.5
798261 911939 0.375
798261 924192 0.125
798261 895481 0.125
798261 943188 0.125
798261 905671 0.125
798261 916840 0.125
798290 906999 0.3333333333333333
798290 880442 0.3333333333333333
798290 853797 0.3333333333333333
798302 893311 0.3333333333333333
798302 861163 0.3333333333333333
798302 836228 0.3333333333333333
798303 823884 1.0
798311 903496 0.3333333333333333
798311 861793 0.3333333333333333
798311 864601 0.3333333333333333


136257

## Construction du graphe final

Construire un DataFrame graphe pour stocker tous les noeuds et tous les liens calculés précédemment. Le dataframe contiendra une colonne 'source' (identifiant du noeud source), une colonne 'destination' et une colonne 'poids'. 
Les colonnes 'source' et 'dest' contiennent à la fois des identifiants utilisateur, chanson et artiste. La colonne 'poids' contient les poids des arcs calculés à partir des poids précédents qui sont multipliés par les coefficients suivants:
* Liens user->artist : 0.5
* Liens user->track: 0.5
* Liens track->track: 1
* Liens artist->track: 1

In [ ]:
userTrack_g = userTrack.selectExpr("userId as source", "trackId as destination", "norm_count * 0.5 as poids")
userArtist_g = userArtist.selectExpr("userId as source", "artistId as destination", "norm_count * 0.5 as poids")
artistTrack_g = artistTrack.selectExpr("artistId as source", "trackId as destination", "norm_count as poids")
trackTrack_g = trackTrack.selectExpr("trackId as source", "trackId2 as destination", "norm_count as poids")

In [ ]:
graphe = userTrack_g.union(userArtist_g).union(artistTrack_g).union(trackTrack_g).persist()

graphe.printSchema()

graphe.count() #résultat: #560759

root
 |-- source: long (nullable = true)
 |-- destination: long (nullable = true)
 |-- poids: double (nullable = true)



560759

### Calcul de recommendation de chansons avec PPR

En utilisant le calul de PageRank Personalisé, recommender à l'utilisateur avec l'identifiant 10 des chansons qu'il n'a pas écoutées.
Rappel de la formule de mise à jour du score de recommendation à chaque itération du calcul:

x[i] = alpha * u[i] + (1-alpha)* sum(xant[j]*poids[j][i])

    - poids[j][i] : poids de l'arc entre j à i
    - u[i]: valeur de personalisation, u[10]=1 et u[i]=0 si i !=10
    - xant[j] : valeur du score du noeud j à l'itération précédente (x0[10]=1 et x0[i]=0 si i !=10)

On considère alpha=0.15 et on effectue le calcul pour 5 itérations (maxiter=5)

### Calcul du vecteur de recommendation x

In [ ]:
import pandas as pd
from pyspark.sql.functions import when

user = 10
d=0.85
alpha=(1-d)
maxiter = 5
# Construire le vecteur d'importance initial
x0  = spark.createDataFrame(pd.DataFrame([(user,1)], columns=["id","rank"]))

print("Importance initiale")
x0.show()

x = x0
for iter in range(maxiter) :

    # nextx = (1-alpha)* sum(xant[j]*poids[j][i])
    nextx = x.join(graphe, x.id == graphe.source).selectExpr('destination', "rank * poids as prod").groupBy('destination').agg(sum('prod').alias('sum_prod'))\
    .select("destination", (col('sum_prod') * d).alias("alpha_sum_prod"))
    
    # x = alpha * u[i] + nextx
    x = nextx.select(col('destination').alias("id"), when(nextx.destination != user, nextx.alpha_sum_prod).otherwise(alpha + nextx.alpha_sum_prod).alias("rank"))#.orderBy('id')
    
    if x.filter(col('id')==user).count() == 0:
      x = x.union(spark.createDataFrame(pd.DataFrame([(user, alpha)], columns=["id","rank"])))

x.persist()
print("Importance finale")
x.show()

x.count() #résultat: 16168

Importance initiale
+---+----+
| id|rank|
+---+----+
| 10|   1|
+---+----+

Importance finale
+------+--------------------+
|    id|                rank|
+------+--------------------+
|810986| 7.18469357712758E-5|
|843951|6.089243971671870...|
|856224|4.285758198937867...|
|879220|1.333570546372265...|
|947737|9.051991059047644E-5|
|938217|5.657976463027959E-6|
|813815|3.379208514585342E-5|
|828085|7.251152666789907...|
|813070|3.707235062801998E-7|
|966496|9.991016395078787E-5|
|892405|6.204220759118849E-5|
|889765| 7.23308841890472E-7|
|919726|3.659091851598001E-7|
|923142|2.706018945187368...|
|863929|9.950739746616022E-7|
|938608|1.093416729222471E-5|
|814505| 4.18054299054014E-5|
|854061|1.781094691069518...|
|843360|2.755065779294699...|
|801947|2.755065779294699...|
+------+--------------------+
only showing top 20 rows



16147


```
# Résultat attendu:

Importance initiale
+---+----+
| id|rank|
+---+----+
| 10| 1|
+---+----+

Importance finale

+------+--------------------+
| id| rank|
+------+--------------------+
|809859|1.078173166785031...|
|811663|2.196173966339472...|
|815398|4.991726499709634E-6|
|819656|2.565325294701734...|
|821585|1.085677623094404...|
|826383|4.623323114868115...|
|828577|3.198389497232263...|
|833948|1.508649194553883E-6|
|834684|8.776578672498915E-6|
|838193|2.702370149734005...|
|839249|1.902150200371450...|
|839622|3.078835227272727E-4|
|839790|2.194482183345908E-7|
|841504| 1.57108458561054E-6|
|844513|5.834083375811894E-8|
|847474|3.237860140464702...|
|853761|8.599848688583445E-4|
|854531|0.001546366379143...|
|856152|6.704348500324493E-8|
|856636|8.663166221940156E-6|
+------+--------------------+
only showing top 20 rows

Out[13]: 16168
```



In [ ]:
x.where(col('id').isin([841504, 844513, 847474, 853761, 854531, 856152, 856636])).show()

+------+--------------------+
|    id|                rank|
+------+--------------------+
|854531|0.001554540512839496|
|856636|9.373849924168476E-6|
|841504|1.696264140221232...|
|847474|3.237165813907451E-5|
|853761|8.599485698358212E-4|
|844513|5.834083375811894E-8|
|856152|6.704348500324492E-8|
+------+--------------------+



### Construction de la liste de chansons non écoutées recommendées
Afficher les 10 chansons les plus recommendées que l'utilisateur n'a pas écoutées, avec leur score
calculé précédemment

In [ ]:
x2 = meta.selectExpr('Name', 'Artist', 'Id as id2')
tracksrec = x.join(x2, x.id == x2.id2).selectExpr('id', 'rank', 'Name', 'Artist')

window = Window.orderBy(col("rank").desc())

tracksreckList = tracksrec.withColumn("position", rank().over(window))\
    .where(col("position")<=10)\
    .orderBy('position').select('id', "rank", 'Name', 'Artist').take(20)   
       
for val in tracksreckList:
   print("%s %s %s %s" % val)

839649 0.04804175618196066 Bazam Bekhand Artist: Mohsen Yeganeh
828318 0.041461989256473215 Unapologetic Bitch Artist: Madonna
960353 0.038525490989983785 Magneto And Titanium Man Artist: Wings
960214 0.03797141587570513 Procession Artist: Queen
955486 0.034484389884170796 Seni Bilen Biliyor Artist: Arif Susam
855194 0.02795106930767553 Porke Artist: Pochill
958924 0.02584262856304979 Pictures Artist: Gemafrei
801772 0.02535247383791928 Celebration Day Artist: Led Zeppelin
823737 0.02194303602829867 Dj Petros Artist: Bang La Decks
807650 0.02194303602829867 Take care Artist: Imany




```
# Résultat attendu:

949111 0.01444789334755391 Hah Heh Hah Artist: Vaya Con Dios
955858 0.014255786458993664 Legz Artist: Jaffa
849768 0.009970446066103254 martin Artist: Dani Martin
803682 0.00532400332583392 Every Morning Artist: Sugar Ray
926933 0.004131316860041196 Who You Want Artist: Qulinez
```



## Calcul des triangles

Implémenter les différentes étapes de l'algorithme amélioré de calcul des triangles 
présenté en cours sur le graphe trackTrack construit précédemment.

In [ ]:
trackTrack.count()

136257

In [ ]:
# Définir une fonction qui prend comme argument une liste d'utilisateurs triés 
# par ordre croissant (users) et retourne une liste de couples ordonnés d'utilisateurs

from pyspark.sql.functions import udf
from pyspark.sql.types import *

def parse_string(users):
    
    results = [[users[i], users[j]] for i in range(len(users)-1) for j in range(i+1, len(users))]
    
    return results

parse_string_udf = udf(parse_string, ArrayType(StringType()))

In [ ]:
# Implémentation de l'algorithme de calcul de triangles

from pyspark.sql.functions import collect_list, sort_array

#Map1 - cours
#Prendre en considération uniquement les couples ordonnés (trackId, track1) (trackId < track1)  
trackOrd = trackTrack.where(col('trackId') < col('trackId2')).select('trackId', 'trackId2').orderBy('trackId', 'trackId2')

#Reduce 1 - cours: Construire pour chaque trackId la liste de couples ordonnés de ses voisins
# a) regrouper les lignes par trackId en construisant la liste de voisins triés par ordre
#    croissant (utiliser sort_array)
neighbors = trackOrd.groupby('trackId').agg(sort_array(collect_list('trackId2')).alias('liste'))

neighbors.show(4)

+-------+--------------------+
|trackId|               liste|
+-------+--------------------+
| 798256|    [923706, 954826]|
| 798258|    [808254, 810685]|
| 798261|[895481, 905671, ...|
| 798290|[853797, 880442, ...|
+-------+--------------------+
only showing top 4 rows



In [ ]:
# b) utiliser la fonction définie précédemment pour retourner la liste de couples de voisins
couples= neighbors.withColumn('liste_couple', explode(parse_string_udf(col('liste')))).select('trackId', 'liste_couple')
couples.show(5)

+-------+----------------+
|trackId|    liste_couple|
+-------+----------------+
| 798256|[923706, 954826]|
| 798258|[808254, 810685]|
| 798261|[895481, 905671]|
| 798261|[895481, 911939]|
| 798261|[895481, 916840]|
+-------+----------------+
only showing top 5 rows



In [ ]:
# Map2 + Reduce 2 - cours
# prendre en considération uniquement les lignes telles que les couples de voisins 
# construits précédemment existent également dans le graphe 
from pyspark.sql.functions import concat, lit, count, desc
trackOrd_couple = trackOrd.withColumn('liste_couple', concat(lit('['), col('trackId'), lit(', '), col('trackId2'), lit(']'))).select('liste_couple')
trackOrd_couple.show()

+----------------+
|    liste_couple|
+----------------+
|[798256, 923706]|
|[798256, 954826]|
|[798258, 808254]|
|[798258, 810685]|
|[798261, 895481]|
|[798261, 905671]|
|[798261, 911939]|
|[798261, 916840]|
|[798261, 924192]|
|[798261, 943188]|
|[798290, 853797]|
|[798290, 880442]|
|[798290, 906999]|
|[798302, 836228]|
|[798302, 861163]|
|[798302, 893311]|
|[798303, 823884]|
|[798311, 861793]|
|[798311, 864601]|
|[798311, 903496]|
+----------------+
only showing top 20 rows



In [ ]:
liste =  couples.join(trackOrd_couple, on='liste_couple')
liste.show(5)

+------------------+-------+
|      liste_couple|trackId|
+------------------+-------+
|[1179685, 1200833]|1176891|
|[1200833, 1218821]|1176891|
|[1225222, 1290141]|1178831|
|[1226775, 1299891]|1197701|
|  [798800, 810775]| 798622|
+------------------+-------+
only showing top 5 rows



In [ ]:
# Calculer le nombre de triangles pour chaque utilisateur et 
# trier le résultat par le nombre de triangles décroissant
triangles = liste.groupBy('trackId').count().orderBy(col('count').desc())

print(triangles.count()) #résultat: 7156
triangles.agg(sum(col("count"))).show() #Résultat: 93400
triangles.orderBy(col("count").desc()).show()

7156
+----------+
|sum(count)|
+----------+
|     93388|
+----------+

+-------+-----+
|trackId|count|
+-------+-----+
| 800288| 1136|
| 808082| 1063|
| 805688|  898|
| 815388|  894|
| 806854|  890|
| 825174|  880|
| 831005|  844|
| 805959|  665|
| 798517|  630|
| 799541|  619|
| 846624|  615|
| 801571|  607|
| 798800|  579|
| 802599|  559|
| 841340|  552|
| 810775|  552|
| 811513|  548|
| 798587|  540|
| 858904|  539|
| 813969|  517|
+-------+-----+
only showing top 20 rows





```
# Résultat attendu:

+-------+------------+
|trackid|nb_triangles|
+-------+------------+
| 800288|        1136|
| 808082|        1063|
| 805688|         898|
| 815388|         893|
| 806854|         890|
| 825174|         881|
| 831005|         844|
| 805959|         665|
| 798517|         630|
| 799541|         619|
| 846624|         615|
| 801571|         607|
| 798800|         579|
| 802599|         559|
| 841340|         552|
| 810775|         552|
| 811513|         548|
| 798587|         541|
| 858904|         539|
| 813969|         517|
+-------+------------+
only showing top 20 rows
```



## Calcul du plus court chemin

Implémenter l'algorithme parallèle de calcul de plus court chemin d'origine fixée vers n'importe quelle destination

In [ ]:
from graphframes.lib import AggregateMessages as AM
from pyspark.sql import functions as F

origin=10
#Rassembler tous les noeuds  dans un seul DF vertices
vertices = graphe.select("source").distinct().withColumnRenamed("source", "id").union(graphe.select("dest").distinct().withColumnRenamed("dest", "id")).distinct() 
# Vecteur des distances de tous les neouds par rapport à origin
distances = vertices.withColumn("distance", F.when(vertices["id"] == origin, 0).otherwise(float("inf")))
active  = spark.createDataFrame(pd.DataFrame([(origin,0)], columns=["idA","distanceA"]))

distances= AM.getCachedDataFrame(distances)

while active.first():

   ....
   active=AM.getCachedDataFrame(...)
   distances=AM.getCachedDataFrame((...)

print("Distances finales:")  
distances.filter(col("distance")!=float('inf')).orderBy(col("distance")).show()

```
# Résultat attendu:

Distances finales:
+------+--------+
|    id|distance|
+------+--------+
|    10|     0.0|
|828318| 0.03125|
|807650| 0.03125|
|801772| 0.03125|
|941064| 0.03125|
|823737| 0.03125|
|824440| 0.03125|
|839649| 0.03125|
|960214| 0.03125|
|855194| 0.03125|
|901153| 0.03125|
|934050| 0.03125|
|943645| 0.03125|
|955486| 0.03125|
|956604| 0.03125|
|958924| 0.03125|
|960353| 0.03125|
|970381| 0.03125|
|976111| 0.03125|
|983989| 0.03125|
+------+--------+
only showing top 20 rows
```